In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import scipy.optimize as sco
import scipy as sp
import statsmodels.stats.correlation_tools as stat
import math
plt.style.use('fivethirtyeight')
#np.random.seed(777)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
#hr is the monthly historical returns
hr=pd.read_csv(r"C:\Users\li_wn\Documents\datafile\00_bwsDomFund_HisRet_202009.csv",\
                        header=0, parse_dates=[0], index_col=0)
#er is the annual expected returns
#make sure hr and er files' columns are matched and aligned with each other
#exp=pd.read_csv(r"C:\Users\s107752\Documents\Optimization\final\test_exp.csv",\
 #                   header=0, parse_dates=[0], index_col=0)
con=pd.read_csv(r"C:\Users\li_wn\Documents\datafile\00_bwsDomAgg_cons.csv",\
                   header=0, parse_dates=[0], index_col=0)
er_table=pd.read_csv(r"C:\Users\li_wn\Documents\datafile\00_bwsDomFund_expRet_202009.csv",\
                    header=0,index_col=0)
er=er_table[0:1]
print(len(er.T))


37


In [28]:
style_exp=er_table[1:9]
asset_exp=er_table[9:25]
print(hr.columns)
print(er.columns)
#print(asset_exp)

Index(['Cash', 'American Funds New World F3', 'Baillie Gifford Emerging Markets Eqs I', 'Columbia Select Large Cap Growth Adv', 'Diamond Hill Large Cap I', 'iShares Core High Dividend ETF', 'iShares Core S&P 500 ETF', 'IVA International I', 'IVA Worldwide I', 'JHancock Disciplined Value I', 'Lazard International Equity Instl', 'Loomis Sayles Growth Y', 'Morgan Stanley Inst Global Franchise I', 'Nuveen Santa Barbara Dividend Growth I', 'Nuveen Small Cap Value I', 'Principal Blue Chip R-6', 'Principal MidCap R-6', 'SPDR S&P Dividend ETF', 'Victory RS Small Cap Growth Y', 'Virtus KAR Small-Cap Value I', 'Baird Aggregate Bond Inst', 'Baird Short-Term Bond Inst', 'Federated Hermes Instl High Yield Bd IS', 'iShares 3-7 Year Treasury Bond ETF', 'iShares Broad USD High Yield Corp Bd ETF', 'iShares Broad USD Invm Grd Corp Bd ETF', 'Janus Henderson Flexible Bond I', 'Janus Henderson Short-Term Bond I', 'Loomis Sayles Investment Grade Bond Y', 'MFS Corporate Bond I',
       'Osterweis Strategic I

In [29]:
#### with exponentially weighted approach ################

business = "AMT SMA tactical"
port="Aggressive"
wgtcon="25Pct"
port_cons="Weight Constraints: Cash=1-10%,Equity=80-100%, FI=0-10%"
port_header=business+' '+port+' Portfolio '+'Benchmark: 100% S&P 500 and 0% AGG'

riskRet_file="C:/Users/li_wn/Documents/output/" + business + "_Asset_riskRet"  + ".xlsx"
corAbs_file="C:/Users/li_wn/Documents/output/" + business + "_AssetCorr_Abs"  + ".xlsx"
covAbs_file="C:/Users/li_wn/Documents/output/" + business + "_AssetCov_Abs"  + ".xlsx"
corRel_file="C:/Users/li_wn/Documents/output/" + business + "_" + port + "_AssetCorr_Rel"  + ".xlsx"
infor_file="C:/Users/li_wn/Documents/output/" + business + "_" + port + "_infoR_" + wgtcon + ".xlsx"
minTE_file="C:/Users/li_wn/Documents//output/" + business + "_" + port + "_minTE_" + wgtcon + ".xlsx"
riskcomp_file="C:/Users/li_wn/Documents/output/" + business + "_" +\
                port + "_"+ wgtcon + "_riskComp.xlsx"
finalcomp_file="C:/Users/li_wn/Documents/output/" + business + "_" +\
                port + "_" + wgtcon + "_finalComp.xlsx"
effWgt_file="C:/Users/li_wn/Documents/output/" + business + "_" +\
                port + "_" + wgtcon + "_effWgt.xlsx"
image_ef="C:/Users/li_wn/Documents/output/" + business + "_" + port + "_" + wgtcon + ".png"
image_wgt="C:/Users/li_wn/DocumentsA/output/" + business + "_" + port + "_" + wgtcon + "_weight.png"




#noa is the number of assets including those in the benchmark
#er are the expected returns for all assets
noa=len(er.T)
h_life=36

# generate asset risk/return table and asset var-covar/correlation matrix
an_vol=np.round(hr.ewm(halflife=h_life).std().tail(1)*np.sqrt(12),2).T
vol=pd.DataFrame(an_vol)
risk_ret=pd.concat([round(er.T,2),vol,round(er.T.iloc[:, 0]/vol.iloc[:, 0],2)],axis=1)
risk_ret.columns=['Exp Ret','Volatility','ExpRet/Vol']
a=hr['S&P 500']
b=hr.drop(columns=['S&P 500'])
nhr=pd.concat([a,b],axis=1)
k=nhr.columns
cr=nhr.ewm(halflife=h_life).corr().tail(noa)
v_cv=nhr.ewm(halflife=h_life).cov().tail(noa)
v_cv=stat.cov_nearest(v_cv)
v_cv=pd.DataFrame(v_cv)
v_cv.columns=k
v_cv.index=k



risk_ret.to_excel(riskRet_file, index = True, header=True)
cr.to_excel(corAbs_file, index = True, header=True)
v_cv.to_excel(covAbs_file,index = True, header=True)

##############################################################################################################
# bw are the weights for benchmark; by default only benchmark constiutents have non-zero weights
# need to change according to different portfolios
bw_wgt=np.array([0.75,0.25])
#bw_wgt=np.array([1])
bw=np.append(np.array(np.repeat(0,noa-2)),bw_wgt)
#bw=np.append(np.array(np.repeat(0,noa-1)),bw_wgt)
#bw=np.array(np.repeat(0,7))
#cw are the weights current portfolio has. It will be used for comparison with optimized portfolio
cw=np.array([0.0105637276346943,1.28658007170756E-15,0.00170290039831731,0.00313079831256747,0.177480069726868,9.31902464467216E-16,4.41652268484174E-16,0.195579065577168,9.44837953024028E-16,0.141832536221124,0.146564586871964,0.105884397131746,0.0171881301465305,0.00578196049023154,0.0958737039213982,0.0539275117231395,0.0422139977892194,0.00035198097590047,2.9861386693926E-16,0.00191039120978829,5.59249057457398E-16,1.62484817195992E-16,5.16459477861662E-16,3.90585445598708E-16,1.42418693463631E-05,3.00806563017326E-16,3.00531019556614E-16,0,0])
#cw=np.array([0.01,0.04,0.1,0.20,0.07,0.15,0.05,0.03,0.03,0.07,0.11,0.04,0.1,0])
#cw=np.array([0.1568,0.2814,0.143,0.0491,0.1835,0.65,0])
###############################################################################################################

# general excess return correlation matrix
bw_hr=hr[['S&P 500', 'Barclays AGG']]
port_hr=hr.drop(columns=['S&P 500', 'Barclays AGG'])
g=np.dot(bw_hr,bw_wgt)
g=pd.DataFrame(g,index=hr.index,columns=['bmkret'])
act_ret=b.sub(g['bmkret'],axis=0)
act_ret_corr=act_ret.corr()
act_ret_corr.to_excel(corRel_file, index = True, header=True)



In [30]:
cov_matrix = hr.ewm(halflife=h_life).cov().tail(noa)
cov_matrix=stat.cov_nearest(cov_matrix)

#5-yr T-bond yield
risk_free_rate = 0.2768

############################################################################################
# Optimization constrainst need to be a dictionary
# cons1 - sum of weight is 
# cons2/cons3 - aggregate active equity/FI holding weight +/-5% of benchmark's
# cons4 - used to minize TE given a certain return target
# cons5 - cash max weight
# cons6 - sum of square of alloc deviation
cons1 = {'type': 'eq', 'fun': lambda x: np.sum(x)-1}
#cons2 = {'type': 'ineq', 'fun': lambda x: 0.1-np.abs(np.sum(x[1:16])-0.9)} # equity
#cons2 = {'type': 'eq', 'fun': lambda x: np.sum(x[1:17])-0}   
#cons3 = {'type': 'ineq', 'fun': lambda x: 0-np.abs(np.sum(x[36:62])-0)}
#cons3 = {'type': 'ineq', 'fun': lambda x: 0.1-np.abs(x[16]-0.1)} # FI
cons4 = {'type': 'ineq', 'fun': lambda x: 0.0-np.abs(x[0]-0.0)} #cash

##############################################################################################

bound=(0,0.25)
bound = tuple(bound for asset in range(noa-2))
bound=bound+((0,0),)+((0,0),)
#bound=((0.1375,0.4125),(0.1375,0.4125),(0.075,0.225),(0.05,0.15),(0.0675,0.2025),(0.0325,0.0975),(0,0))

#bound=bound+((0,0),)*7+((0,1),)+((0,1),)

In [31]:
# define various risk and return metrics of a portfolio
# er is dataframe;weights, bw, cov_matrix are arrays; 
# Therfore, std/te/beta are floats, and returns/infor/a_er/sr are arrays
def portfolio_annualised_performance(weights, bw, er, cov_matrix):
    returns = np.dot(er,weights)#np.sum(er*weights ) ##*252
    b_er=np.dot(er,bw)
    a_er=returns-b_er
    aw=weights-bw 
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(12)  ##np.sqrt(252)
    te=np.sqrt(np.dot(aw.T, np.dot(cov_matrix, aw))) * np.sqrt(12)
    infor=a_er/te
    beta=np.dot(bw.T, np.dot(cov_matrix, weights))/np.dot(bw.T, np.dot(cov_matrix, bw))
    sr=(returns-risk_free_rate)/std
    hhi=np.sum(np.square(weights))
    port_sty=np.dot(style_exp,weights)
    port_sty=pd.DataFrame(port_sty,index=style_exp.index,columns=["PortExp"])
    port_asset_exp=np.dot(asset_exp,weights)
    port_asset_exp=pd.DataFrame(port_asset_exp,index=asset_exp.index,columns=["PortExp"])
    diff_exp=port_asset_exp.sub(con,axis=1)
    diff_exp=diff_exp['PortExp']
    diff_exp=diff_exp.to_numpy()
    diff_exp=np.sum(np.square(diff_exp))
    
    return std, returns,te,infor,beta,a_er,sr,hhi,port_sty,port_asset_exp, diff_exp

In [32]:
#def port_style(weights):
    #return portfolio_annualised_performance(weights, bw, er, cov_matrix)[10]
#cons6={'type': 'ineq', 'fun': lambda x: 0.04-port_style(x)}

def port_asset_exposure(weights):
    return portfolio_annualised_performance(weights, bw, er, cov_matrix)[9]
cons5={'type': 'ineq', 'fun': lambda x: port_asset_exposure(x).loc["R.1000 V"]-port_asset_exposure(x).loc["R.1000 G"]*1.1}
cons6={'type': 'ineq', 'fun': lambda x: 0.10-port_asset_exposure(x).loc["HY USA"]}
cons7={'type': 'ineq', 'fun': lambda x: port_asset_exposure(x).loc["R.1000 G"]-port_asset_exposure(x).loc["R.MidCap"]*1.1}
cons8={'type': 'ineq', 'fun': lambda x: port_asset_exposure(x).loc["R.1000 G"]-port_asset_exposure(x).loc["R.2000"]*1.1}


#cons_reg=(cons1,cons2,cons3,cons4)
#cons_asset=(cons5,cons6)
#cons_reg_asset=(cons1,cons2,cons3,cons4,cons5,cons6,cons7,cons8)
#FI portfolio only
#cons_reg_asset=(cons1,cons2,cons3,cons4)#,cons6)#cons5,cons6,cons7,cons8)
cons_reg_asset=(cons1,cons4)#cons2,cons3,
def neg_info_ratio(weights, er, cov_matrix, risk_free_rate):
    infor = portfolio_annualised_performance(weights, bw, er, cov_matrix)[3]
    return -infor

def max_info_ratio(er, bw, cov_matrix, risk_free_rate):
    num_assets = len(er.T)
    args = (er, cov_matrix, risk_free_rate)
    #constraints = (cons1,cons2,cons3,cons5)
    #constraints = (cons1,cons5)#,cons6)
    result = sco.minimize(neg_info_ratio, noa*[1./noa,], args=args,
                        method='SLSQP', bounds=bound, constraints=cons_reg_asset)
    return result

In [33]:
def portfolio_te(weights, er, cov_matrix):
    return portfolio_annualised_performance(weights, bw, er, cov_matrix)[2]


def min_trackingerror(er, bw,cov_matrix):
    num_assets = len(er.T)
    args = (er, cov_matrix)
    #constraints = (cons1,cons2,cons3,cons5,cons7)
    #constraints = (cons1,cons5)#,cons6)
    result = sco.minimize(portfolio_te, noa*[1./noa,], args=args,
                        method='SLSQP', bounds=bound, constraints=cons_reg_asset)

    return result

In [34]:
def efficient_return(er, bw,cov_matrix, target):
    num_assets = len(er.T)
    args = (er, cov_matrix)

    def portfolio_return(weights,bw):
        return portfolio_annualised_performance(weights,bw,er, cov_matrix)[5]

    cons0 = {'type': 'eq', 'fun': lambda x: portfolio_return(x,bw) - target}   
    
    #constraints = (cons1,cons2,cons3,cons4,cons5,cons7)
    #cons_total=(cons1,cons2,cons3,cons4,cons5,cons6,cons7,cons8,cons0)
    #cons_total=(cons1,cons2,cons3,cons4,cons0)#cons6,cons0)
    cons_total=(cons1,cons4,cons0)#cons2,cons3,
    result = sco.minimize(portfolio_te, noa*[1./noa,], args=args, method='SLSQP', bounds=bound, constraints=cons_total)
    return result


def efficient_frontier(er, bw,cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(er,bw,cov_matrix, ret))
    return efficients

In [35]:
##############################################
def efficient_te(er, bw,cov_matrix, target_te):
    args = (er, cov_matrix)

    def portfolio_infor(weights,er, cov_matrix):
        return -portfolio_annualised_performance(weights,bw,er, cov_matrix)[5]
    def portfolio_te(weights, er, cov_matrix):
        return portfolio_annualised_performance(weights, bw, er, cov_matrix)[2]

    #cons4 = {'type': 'eq', 'fun': lambda x: portfolio_te(x,er,cov_matrix) - target_te}   
    #constraints = (cons1,cons2,cons3,cons4,cons5,cons7)
    #cons_total=(cons1,cons2,cons3,cons4,cons5,cons6,cons7,cons8,cons0)
    #cons_total=(cons1,cons2,cons3,cons4,cons0)#cons6,cons0)
    cons_total=(cons1,cons4,cons0)#cons2,cons3,
    result = sco.minimize(portfolio_infor, noa*[1./noa,], args=args, method='SLSQP', bounds=bound, constraints=cons_total)
    return result

In [36]:
def full_ef(er, bw,cov_matrix, risk_free_rate):
    pd.set_option('display.width',1000)
    #results= random_portfolios(num_portfolios,er, bw,cov_matrix, risk_free_rate)[0]
    bwVol=np.sqrt(np.dot(bw.T, np.dot(cov_matrix, bw))) * np.sqrt(12)
    max_infor = max_info_ratio(er, bw, cov_matrix, risk_free_rate)
    sdp, rp,te,ir,beta,ar,sr,hhi,port_sty,port_asset_exp,port_exp =\
    portfolio_annualised_performance(max_infor['x'], bw,er, cov_matrix)
    
    
    aw=max_infor.x-bw
    max_infor_prc=np.multiply(np.dot(cov_matrix, max_infor.x),max_infor.x.T)*12/sdp
    max_infor_prc_pct=max_infor_prc/np.sum(max_infor_prc)
    max_infor_prc = pd.DataFrame(max_infor_prc,index=hr.columns,columns=['riskCon'])
    max_infor_prc.riskCon=[round(i,2)for i in max_infor_prc.riskCon]
   # max_infor_prc=max_infor_prc.T
   
    max_infor_prc_pct = pd.DataFrame(max_infor_prc_pct,index=hr.columns,columns=['riskConPct'])
    max_infor_prc_pct.riskConPct = [round(i*100,2)for i in max_infor_prc_pct.riskConPct]
    #max_infor_prc_pct = max_infor_prc_pct.T

    max_infor_arc=np.multiply(np.dot(cov_matrix, aw),aw.T)*12/te
    max_infor_arc_pct=max_infor_arc/np.sum(max_infor_arc)
    max_infor_arc = pd.DataFrame(max_infor_arc,index=hr.columns,columns=['actRiskCon'])
    max_infor_arc.actRiskCon=[round(i,2)for i in max_infor_arc.actRiskCon]

    max_infor_arc_pct = pd.DataFrame(max_infor_arc_pct,index=hr.columns,columns=['actRiskConPct'])
    max_infor_arc_pct.actRiskConPct = [round(i*100,2)for i in max_infor_arc_pct.actRiskConPct]
    
    max_infor_alloc = pd.DataFrame(max_infor.x,index=hr.columns,columns=['alloc'])
    max_infor_alloc.alloc = [round(i*100,2)for i in max_infor_alloc.alloc]
    #max_infor_alloc = max_infor_alloc.T

    final_infor=pd.concat([max_infor_alloc, max_infor_prc,max_infor_prc_pct,
                            max_infor_arc,max_infor_arc_pct], axis=1)

    
    min_te = min_trackingerror(er, bw,cov_matrix)
    sdp_min, rp_min, te_min,ir_min,beta_min,ar_min,sr_min,hhi_min,port_sty_min,port_asset_exp_min,port_exp_min= \
    portfolio_annualised_performance(min_te['x'], bw,er, cov_matrix)
    
    aw=min_te.x-bw
    min_te_prc=np.multiply(np.dot(cov_matrix, min_te.x),min_te.x.T)*12/sdp_min
    min_te_prc_pct=min_te_prc/np.sum(min_te_prc)
    min_te_prc = pd.DataFrame(min_te_prc,index=hr.columns,columns=['riskCon'])
    min_te_prc.riskCon=[round(i,2)for i in min_te_prc.riskCon]
   # min_te_prc=min_te_prc.T
   
    min_te_prc_pct = pd.DataFrame(min_te_prc_pct,index=hr.columns,columns=['riskConPct'])
    min_te_prc_pct.riskConPct = [round(i*100,2)for i in min_te_prc_pct.riskConPct]
    #min_te_prc_pct = min_te_prc_pct.T

    min_te_arc=np.multiply(np.dot(cov_matrix, aw),aw.T)*12/te_min
    min_te_arc_pct=min_te_arc/np.sum(min_te_arc)
    min_te_arc = pd.DataFrame(min_te_arc,index=hr.columns,columns=['actRiskCon'])
    min_te_arc.actRiskCon=[round(i,2)for i in min_te_arc.actRiskCon]

    min_te_arc_pct = pd.DataFrame(min_te_arc_pct,index=hr.columns,columns=['actRiskConPct'])
    min_te_arc_pct.actRiskConPct = [round(i*100,2)for i in min_te_arc_pct.actRiskConPct]
    
    min_te_alloc = pd.DataFrame(min_te.x,index=hr.columns,columns=['alloc'])
    min_te_alloc.alloc = [round(i*100,2)for i in min_te_alloc.alloc]
    #min_te_alloc = min_te_alloc.T

    final_te=pd.concat([min_te_alloc, min_te_prc,min_te_prc_pct,
                            min_te_arc,min_te_arc_pct], axis=1)
    
    

    #########################################################################################
    an_vol = np.std(hr) * np.sqrt(12)
    an_rt = er
    ############################################################################################
    print("-"*80)
    #print("Weight Constraints: Equity<=20%, FI<=25%")
    #print("Asset Class Active Weight Constraints: +/-5%")
    print(port_header)
    print(port_cons)
    print("-"*80)
    print("Maximum Information Ratio Portfolio Allocation\n")
    print("Portfolio Return:", round(rp[0],2))
    print("Portfolio Volatility:", round(sdp,2))
    print("Benchmark Volatility:",round(bwVol,2))
    print("Active Return:",round(ar[0],2))
    print("Tracking Error:", round(te,2))
    print("Information Ratio:", round(ir[0],2))
    print("Beta:", round(beta,2))
    print("Sharpe Ratio:",round(sr[0],2))
    print("Herfindahl-Hirschman Index (HHI):",round(hhi,2))
    print("\n")
    print(final_infor)
    #print(max_sharpe_alloc)
    #print(max_sharpe_prc)
    #print(max_sharpe_prc_pct)
    print("-"*80)
    print(port_header)
    print(port_cons)
    print("-"*80)
    print("Minimum Tracking Error Portfolio Allocation\n")
    print("Portfolio Return:", round(rp_min[0],2))
    print("Portfolio Volatility:", round(sdp_min,2))
    print("Benchmark Volatility:",round(bwVol,2))
    print("Active Return:",round(ar_min[0],2))
    print("Tracking Error:", round(te_min,2))
    print("Information Ratio:", round(ir_min[0],2))
    print("Beta:", round(beta_min,2))
    print("Sharpe Ratio:",round(sr_min[0],2))
    print("Herfindahl-Hirschman Index (HHI):",round(hhi_min,2))
    print("\n")
    print(final_te)
    print("\n")
    print("-"*80)
    
    print("Asset Returns and Volatility\n")
    #for i, txt in enumerate(hr.columns):
    #  print(txt,":","Exp Ret",round(an_rt.iat[0,i],2),", Volatility:",round(an_vol[i],2),", Ret/Vol:",\
    #          round(an_rt.iat[0,i]/an_vol[i],2))
    print(risk_ret)
    print("-"*80)
    
    fig, ax = plt.subplots(figsize=(12, 9))
    
    #ax.scatter(an_vol,an_rt,marker='o',s=200)
    
       
    #for i, txt in enumerate(hr.columns):
      
        #ax.annotate(txt, (an_vol[i],an_rt.iat[0,i]), xytext=(10,0), textcoords='offset points')
    ax.scatter(te,ar,marker='*',color='r',s=500, label='Maximum Information ratio')
    ax.scatter(te_min,ar_min,marker='*',color='g',s=500, label='Minimum TE')
    
    #############################
    target = np.linspace(ar_min,ar,30)
    efficient_portfolios = efficient_frontier(er, bw,cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='blue', label='efficient frontier')
    #ax.scatter(te_cw,ar_cw,marker='o',color='g',s=100)
    #plt.plot((te_cw,te_minte_cw),(ar_cw,ar_cw),color='green', linestyle='dashed',linewidth=2)
    #plt.plot((te_cw,te_cw),(ar_cw,ar_maxret_cw),color='green', linestyle='dashed',linewidth=2)
    plt.title(business+' '+port+' '+'Portfolio Efficient Frontier '+wgtcon)
    plt.xlabel('Tracking Error')
    plt.ylabel('Active Return')
    plt.legend(labelspacing=0.8)
   
    
    plt.savefig(image_ef)
    
    ef_wgt=[w['x']for w in efficient_portfolios]
    ef_wgt=pd.DataFrame(ef_wgt)
    ef_wgt=ef_wgt.T
    ef_te=[p['fun'] for p in efficient_portfolios]
    ef_wgt.index=hr.columns
    ef_wgt.columns=ef_te
    
    port_style=pd.DataFrame()
    port_asset=pd.DataFrame()
    port_attr=pd.DataFrame()
    temp=np.shape(8)
    port_div=np.empty(0)
    for w in efficient_portfolios:
        temp=portfolio_annualised_performance(w['x'], bw,er, cov_matrix)[0:8]
        temp=np.array(temp,dtype=float)
        temp=pd.DataFrame(temp)
        port_attr=pd.concat([port_attr,temp],axis=1)
        port_style=pd.concat([port_style,portfolio_annualised_performance(w['x'], bw,er, cov_matrix)[8]],axis=1)
        port_asset=pd.concat([port_asset,portfolio_annualised_performance(w['x'], bw,er, cov_matrix)[9]*100],axis=1)
        port_div=np.append(port_div,portfolio_annualised_performance(w['x'], bw,er, cov_matrix)[10])
        
        

    port_style.columns=ef_te
    port_asset.columns=ef_te
    port_div=pd.DataFrame(port_div).T
    port_div.columns=ef_te
    port_div.index=['alloc_deviation']
    port_attr.columns=ef_te
    port_attr.index=['Portfolio Volatility','Portfolio Return','Tracking Error','Information Ratio','Beta',\
                    'Active Return','Sharpe Ratio','Herfindahl-Hirschman Index (HHI)']
    
    port_prof=pd.concat([port_attr,ef_wgt,port_style,port_asset,port_div],axis=0)
    
        
    col=sns.color_palette("hls", noa)
    
    labels=pd.DataFrame(hr.columns)
    labels=labels.drop(labels.index[-1])
    fig, ax = plt.subplots(figsize=(12, 10.5))

    ax.stackplot(ef_te, ef_wgt*100,colors=col)
    ax.legend(hr.columns,loc='upper center', bbox_to_anchor=(0.5, -0.08),fancybox=True, shadow=True, ncol=3)
    #ax.margins(0,0)
    plt.title(business+' '+port+' '+'Efficient Portfolios Asset Weights '+wgtcon)
    plt.xlabel('Tracking Error')
    plt.ylabel('Asset Weights')
    #plt.legend(labelspacing=0.8)
    plt.savefig(image_wgt)
   
    
    final_infor.to_excel(infor_file, index = True, header=True)
    final_te.to_excel(minTE_file, index = True, header=True)
    #riskcompare.to_excel(riskcomp_file,index=True, header=True)
    #final_comp.to_excel(finalcomp_file,index=True,header=True)
    #ef_wgt.to_excel(effWgt_file,index=True,header=True)
    port_prof.to_excel(effWgt_file,index=True,header=True)